# 第8章 分类数据

In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


## 一、category的创建及其性质
### 1. 分类变量的创建
#### （a）用Series创建

In [2]:
pd.Series(["a", "b", "c", "a"], dtype="category")

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### （b）对DataFrame指定类型创建

In [3]:
temp_df = pd.DataFrame({'A':pd.Series(["a", "b", "c", "a"], dtype="category"),'B':list('abcd')})
temp_df.dtypes

A    category
B      object
dtype: object

#### （c）利用内置Categorical类型创建

In [4]:
cat = pd.Categorical(["a", "b", "c", "a"], categories=['a','b','c'])
pd.Series(cat)

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): [a, b, c]

#### （d）利用cut函数创建

#### 默认使用区间类型为标签

In [5]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60])

[(10, 30], (0, 10], (10, 30], (30, 60], (30, 60]]
Categories (3, interval[int64]): [(0, 10] < (10, 30] < (30, 60]]

#### 可指定字符为标签

In [6]:
pd.cut(np.random.randint(0,60,5), [0,10,30,60], right=False, labels=['0-10','10-30','30-60'])

[10-30, 30-60, 30-60, 10-30, 30-60]
Categories (3, object): [0-10 < 10-30 < 30-60]

### 2. 分类变量的结构
#### 一个分类变量包括三个部分，元素值（values）、分类类别（categories）、是否有序（order）
#### 从上面可以看出，使用cut函数创建的分类变量默认为有序分类变量
#### 下面介绍如何获取或修改这些属性
#### （a）describe方法
#### 该方法描述了一个分类序列的情况，包括非缺失值个数、元素值类别数（不是分类类别数）、最多次出现的元素及其频数

In [7]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.describe()

count     4
unique    3
top       a
freq      2
dtype: object

#### （b）categories和ordered属性
#### 查看分类类别和是否排序

In [8]:
s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [9]:
s.cat.ordered

False

### 3. 类别的修改

#### （a）利用set_categories修改
#### 修改分类，但本身值不会变化

In [10]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.set_categories(['new_a','c'])

0    NaN
1    NaN
2      c
3    NaN
4    NaN
dtype: category
Categories (2, object): [new_a, c]

在这里为什么更改类型之后里面的元素值是改变了的，并且为什么只改变了这里面abca中的aba，没有改变c?  
因为他在这里更改了类型，但是c的类型还是没有变，所以前面的那个c就不会变成nan，其他的就会变成nan，因为原先的类型变了。

#### （b）利用rename_categories修改
#### 需要注意的是该方法会把值和分类同时修改

In [11]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.rename_categories(['new_%s'%i for i in s.cat.categories])

0    new_a
1    new_b
2    new_c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, new_c, new_d]

如果后面没有前面相应的值，就不会被创建出来。

#### 利用字典修改值

In [12]:
s.cat.rename_categories({'a':'new_a','b':'new_b'})

0    new_a
1    new_b
2        c
3    new_a
4      NaN
dtype: category
Categories (4, object): [new_a, new_b, c, d]

#### （c）利用add_categories添加

In [13]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.add_categories(['e'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (5, object): [a, b, c, d, e]

#### （d）利用remove_categories移除

In [14]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_categories(['d'])

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

#### （e）删除元素值未出现的分类类型

In [15]:
s = pd.Series(pd.Categorical(["a", "b", "c", "a",np.nan], categories=['a','b','c','d']))
s.cat.remove_unused_categories()

0      a
1      b
2      c
3      a
4    NaN
dtype: category
Categories (3, object): [a, b, c]

## 二、分类变量的排序
#### 前面提到，分类数据类型被分为有序和无序，这非常好理解，例如分数区间的高低是有序变量，考试科目的类别一般看做无序变量

### 1. 序的建立

#### （a）一般来说会将一个序列转为有序变量，可以利用as_ordered方法

In [16]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.as_ordered()
s

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

#### 退化为无序变量，只需要使用as_unordered

In [17]:
s.cat.as_unordered()

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a, c, d]

#### （b）利用set_categories方法中的order参数

In [18]:
pd.Series(["a", "d", "c", "a"]).astype('category').cat.set_categories(['a','c','d'],ordered=True)

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

#### （c）利用reorder_categories方法
#### 这个方法的特点在于，新设置的分类必须与原分类为同一集合

In [19]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s.cat.reorder_categories(['a','c','d'],ordered=True)

0    a
1    d
2    c
3    a
dtype: category
Categories (3, object): [a < c < d]

In [20]:
#s.cat.reorder_categories(['a','c'],ordered=True) #报错
#s.cat.reorder_categories(['a','c','d','e'],ordered=True) #报错

### 2. 排序

#### 先前在第1章介绍的值排序和索引排序都是适用的

In [21]:
s = pd.Series(np.random.choice(['perfect','good','fair','bad','awful'],50)).astype('category')
s.cat.set_categories(['perfect','good','fair','bad','awful'][::-1],ordered=True).head()

0       good
1       fair
2        bad
3    perfect
4    perfect
dtype: category
Categories (5, object): [awful < bad < fair < good < perfect]

In [22]:
s.sort_values(ascending=False).head()

29    perfect
17    perfect
31    perfect
3     perfect
4     perfect
dtype: category
Categories (5, object): [awful, bad, fair, good, perfect]

In [23]:
df_sort = pd.DataFrame({'cat':s.values,'value':np.random.randn(50)}).set_index('cat')
df_sort.head()

,value
cat,
good,-1.746975
fair,0.836732
bad,0.094912
perfect,-0.724338
perfect,-1.456362


In [24]:
df_sort.sort_index().head()

,value
cat,
awful,0.245782
awful,0.063991
awful,1.541862
awful,-0.062976
awful,0.472542


## 三、分类变量的比较操作

### 1. 与标量或等长序列的比较

#### （a）标量比较

In [25]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == 'a'

0     True
1    False
2    False
3     True
dtype: bool

#### （b）等长序列比较

In [26]:
s == list('abcd')

0     True
1    False
2     True
3    False
dtype: bool

### 2. 与另一分类变量的比较

#### （a）等式判别（包含等号和不等号）
#### 两个分类变量的等式判别需要满足分类完全相同

In [27]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
s == s

0    True
1    True
2    True
3    True
dtype: bool

In [28]:
s != s

0    False
1    False
2    False
3    False
dtype: bool

In [29]:
s_new = s.cat.set_categories(['a','d','e'])
#s == s_new #报错

#### （b）不等式判别（包含>=,<=,<,>）
#### 两个分类变量的不等式判别需要满足两个条件：① 分类完全相同 ② 排序完全相同

In [30]:
s = pd.Series(["a", "d", "c", "a"]).astype('category')
#s >= s #报错

In [31]:
s = pd.Series(["a", "d", "c", "a"]).astype('category').cat.reorder_categories(['a','c','d'],ordered=True)
s >= s

0    True
1    True
2    True
3    True
dtype: bool

## 四、问题与练习

#### 【问题一】 如何使用union_categoricals方法？它的作用是什么？

pandas.api.types.union_categoricals
1
`#将类别合并
a = pd.Categorical(["b", "c"])
b = pd.Categorical(["a", "b"])
union_categoricals([a, b])
[b, c, a, b]
Categories (3, object): [b, c, a]`

#### 1.4.0.2  【问题二】 利用concat方法将两个序列纵向拼接，它的结果一定是分类变量吗？什么情况下不是？
不一定，只有分类的数量和类别一样，才是分类变量

#### 1.4.0.3  【问题三】 当使用groupby方法或者value_counts方法时，分类变量的统计结果和普通变量有什么区别？


In [4]:
df = pd.read_csv('data/table.csv')
df.head()

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+


In [5]:
df['Physics'].value_counts()  # 12ms

B+    9
B     8
B-    6
A     4
A+    3
A-    3
C     2
Name: Physics, dtype: int64

In [6]:
df['Physics'].astype('category').value_counts() # 26ms

B+    9
B     8
B-    6
A     4
A-    3
A+    3
C     2
Name: Physics, dtype: int64

In [7]:
df['Physics']=df['Physics'].astype('category')
for name ,group in df.groupby('Physics'):
    print(name)
    display(group)

A


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
13,S_1,C_3,1304,M,street_2,195,70,85.2,A
19,S_2,C_1,2105,M,street_4,170,81,34.2,A
26,S_2,C_3,2302,M,street_5,171,88,32.7,A
30,S_2,C_4,2401,F,street_2,192,62,45.3,A


A+


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
0,S_1,C_1,1101,M,street_1,173,63,34.0,A+
7,S_1,C_2,1203,M,street_6,160,53,58.8,A+
22,S_2,C_2,2203,M,street_4,155,91,73.8,A+


A-


...

,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
1,S_1,C_1,1102,F,street_2,192,73,32.5,B+
2,S_1,C_1,1103,M,street_2,186,82,87.2,B+
4,S_1,C_1,1105,F,street_4,159,64,84.8,B+
10,S_1,C_3,1301,M,street_4,161,68,31.5,B+
16,S_2,C_1,2102,F,street_6,161,61,50.6,B+
18,S_2,C_1,2104,F,street_5,159,97,72.2,B+
21,S_2,C_2,2202,F,street_7,194,77,68.5,B+
25,S_2,C_3,2301,F,street_4,157,78,72.3,B+
32,S_2,C_4,2403,F,street_6,158,60,59.7,B+


B-


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
3,S_1,C_1,1104,F,street_2,167,81,80.4,B-
6,S_1,C_2,1202,F,street_4,176,94,63.5,B-
9,S_1,C_2,1205,F,street_6,167,63,68.4,B-
14,S_1,C_3,1305,F,street_5,187,69,61.7,B-
17,S_2,C_1,2103,M,street_4,157,61,52.5,B-
23,S_2,C_2,2204,M,street_1,175,74,47.2,B-


C


,School,Class,ID,Gender,Address,Height,Weight,Math,Physics
15,S_2,C_1,2101,M,street_7,174,84,83.3,C
27,S_2,C_3,2303,F,street_7,190,99,65.9,C


#### 1.4.0.4  【问题四】 下面的代码说明了Series创建分类变量的什么“缺陷”？如何避免？（提示：使用Series中的copy参数）

In [32]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat")
cat

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

In [33]:
s.iloc[0:2] = 10
cat

[10, 10, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

问题应该是我们对于Series里面的内容进行改变时，Categories中相对应的内容也进行了改变，  
避免的方式时在创建Series时copy 参数设为True这样一来就会自动拷贝一份输入的Categories,  
这时候我i们再对Series进行改变，Categories中相对应的内容不变。  
copy：一个布尔值。如果为True，则拷贝输入数据data


In [9]:
cat = pd.Categorical([1, 2, 3, 10], categories=[1, 2, 3, 4, 10])
s = pd.Series(cat, name="cat",copy=True)
cat # 用cat作为分类创建series，改series的时候，cat的也变了

[1, 2, 3, 10]
Categories (5, int64): [1, 2, 3, 4, 10]

#### 【练习一】 现继续使用第四章中的地震数据集，请解决以下问题：
#### （a）现在将深度分为七个等级：[0,5,10,15,20,30,50,np.inf]，请以深度等级Ⅰ,Ⅱ,Ⅲ,Ⅳ,Ⅴ,Ⅵ,Ⅶ为索引并按照由浅到深的顺序进行排序。
#### （b）在（a）的基础上，将烈度分为4个等级：[0,3,4,5,np.inf]，依次对南部地区的深度和烈度等级建立多级索引排序。

In [34]:
pd.read_csv('data/Earthquake.csv').head()

,日期,时间,维度,经度,方向,距离,深度,烈度
0,2003.05.20,12:17:44 AM,39.04,40.38,west,0.1,10.0,0.0
1,2007.08.01,12:03:08 AM,40.79,30.09,west,0.1,5.2,4.0
2,1978.05.07,12:41:37 AM,38.58,27.61,south_west,0.1,0.0,0.0
3,1997.03.22,12:31:45 AM,39.47,36.44,south_west,0.1,10.0,0.0
4,2000.04.02,12:57:38 AM,40.80,30.24,south_west,0.1,7.0,0.0


In [23]:
# a
df = pd.read_csv('data/Earthquake.csv')
df.head()
df_a = df.copy()
df_a['深度'] = pd.cut(df_a['深度'], [-1e-10,5,10,15,20,30,50,np.inf],labels=['Ⅰ','Ⅱ','Ⅲ','Ⅳ','Ⅴ','Ⅵ','Ⅶ'])
df_a.set_index('深度').sort_index().head()

,日期,时间,维度,经度,方向,距离,烈度
深度,,,,,,,
Ⅰ,2009.09.09,12:54:13 AM,42.42,43.03,north_east,95.4,0.0
Ⅰ,1997.06.16,12:18:04 AM,37.92,29.17,north_east,3.2,0.0
Ⅰ,2011.10.25,12:29:45 AM,38.96,43.64,south_east,1.6,3.9
Ⅰ,1995.07.23,12:05:04 AM,37.61,29.29,north_east,3.2,0.0
Ⅰ,2013.06.10,12:39:19 AM,38.53,43.85,south_east,1.6,3.7


In [24]:
# b
df_a['烈度'] = pd.cut(df_a['烈度'], [-1e-10,3,4,5,np.inf],labels=['Ⅰ','Ⅱ','Ⅲ','Ⅳ'])
df_a.set_index(['深度','烈度']).sort_index().head()

日期           时间     维度     经度          方向   距离
深度 烈度                                                        
Ⅰ  Ⅰ   1978.05.07  12:41:37 AM  38.58  27.61  south_west  0.1
   Ⅰ   2000.02.07  12:11:45 AM  40.05  34.07  south_east  0.1
   Ⅰ   1971.05.20  12:08:46 AM  37.72  30.00  north_east  0.1
   Ⅰ   1985.01.28  12:20:56 AM  38.85  29.06  north_east  0.1
   Ⅰ   1990.07.05  12:43:04 AM  37.87  29.18        east  0.1

#### 【练习二】 对于分类变量而言，调用第4章中的变形函数会出现一个BUG（目前的版本下还未修复）：例如对于crosstab函数，按照[官方文档的说法](https://pandas.pydata.org/pandas-docs/version/1.0.0/user_guide/reshaping.html#cross-tabulations)，即使没有出现的变量也会在变形后的汇总结果中出现，但事实上并不是这样，比如下面的例子就缺少了原本应该出现的行'c'和列'f'。基于这一问题，请尝试设计my_crosstab函数，在功能上能够返回正确的结果。

In [35]:
foo = pd.Categorical(['a', 'b'], categories=['a', 'b', 'c'])
bar = pd.Categorical(['d', 'e'], categories=['d', 'e', 'f'])
pd.crosstab(foo, bar)

col_0,d,e
row_0,,
a,1,0
b,0,1
